### Sources:
* https://www.kaggle.com/cpmpml/spell-checker-using-word2vec
* http://norvig.com/spell-correct.html

In [1]:
from gensim.models.word2vec import Word2Vec
from nltk import edit_distance

C:\Users\Atul\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
model = Word2Vec.load("D:\\Projects\\theGuardian\\wvguardian_v01")
WORDS = {word: i for i,word in enumerate(model.wv.index2word)}

In [15]:
def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or known(edits3(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def edits3(word): 
    "All edits that are three edits away from `word`."
    return (e3 for e2 in edits2(word) for e3 in edits1(e2))

In [14]:
correction('maus')

'mass'

In [16]:
correction('duscroptoin')

'description'

In [17]:
correction('liverfol')

'liverpool'

### Attempt #2

Attempting to build on the previous implementation.

#### Pros
1. This implementation works much faster when the edit distance is >2
2. Can specify the maximum distance between the `word` and its `correction`

#### Cons
1. Slower than previous implementation when edit distance is <=2

In [60]:
def get_distance(word, distance):
    return list(filter(lambda a: a[1]<= min(distance,len(word)/2), [(x, edit_distance(x, word)) for x in WORDS.keys() \
            if len(x)<=len(word)+2 and len(x)>=len(word)-2]))

def get_candidates(word_list, distance):
    try:
        return list(list(zip(*list(filter(lambda x: x[1]==distance, word_list))))[0])
    except:
        return []

def candidates(word, distance):
    word_list = get_distance(word, distance)
    for i in range(distance+1):
        candidate = get_candidates(word_list,i)
        if candidate:
            return candidate
    return [word]

def P(candidate): 
    "Probability of `word`."
    # use inverse of rank as proxy returns 0 if the word isn't in the dictionary
    return - WORDS.get(candidate, 0)

def correction(word, max_distance=3):
    "Most probable spelling correction for word."
    return max(candidates(word, distance=max_distance), key=P)

In [51]:
to_correct = 'maus'
correction('maus')

'mass'

In [61]:
to_correct = 'duscriptoine'
correction(to_correct)

returning original word


'duscriptoine'

In [41]:
%%timeit
to_correct = 'liverfl'
correction(to_correct)

2.6 s ± 5.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
to_correct = 'liverfl'
candidates(to_correct)

{'liberal', 'literal', 'liver'}

#### Testing the spellchecker

In [73]:
import requests
import time

In [74]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [84]:
spelltest(Testset(requests.get("http://norvig.com/spell-testset1.txt").text.splitlines()))

returning original word
returning original word
returning original word
returning original word
returning original word
returning original word
returning original word
returning original word
returning original word
returning original word
54% of 270 correct (29% unknown) at 0 words per second 
